In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer#,TfidfTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os

In [2]:
ad_feature=pd.read_csv('../data/adFeature.csv')
user_feature=pd.read_csv('../data/userFeature/userFeature_2.csv')#29
train=pd.read_csv('../data/train/train_1.csv')
predict=pd.read_csv('../data/test1/test1_1.csv')

In [3]:
train.loc[train['label']==-1,'label']=0
predict['label']=-1
data=pd.concat([train,predict])
data=pd.merge(data,ad_feature,on='aid',how='left')
data=pd.merge(data,user_feature,on='uid',how='left')
data=data.fillna('-1')

#data_label1 = data[data.label == 1]

In [5]:
#转化数值，减少内存
obj2uint8=['label','advertiserId','campaignId','creativeId','creativeSize','adCategoryId','productId','productType','age','gender','education','consumptionAbility','carrier','house']
for feature in obj2uint8:
    #print(feature+' is transformed to interger num...')
    data[feature]=data[feature].astype('uint8')

#print('LBS is transformed to interger num...')
data['LBS']=data['LBS'].astype('uint16')

print('data has been reshaped OK!')

data has been reshaped OK!


In [6]:
# 处理联网类型特征,多值->对应位标记为1

#因为只读了部分userFeature,所以做连接没连到的值会变成'-1'，这里暂且先替换为'0'。   
#遍历全部userFeature连接后是不会有'-1'的,就可以删掉下面这行
data.loc[ (data.ct == '-1' ), 'ct' ] = '0'

ct = data['ct'].values
ct = [m.split(' ') for m in ct]
cts = []
for i in ct:
    index = [0, 0, 0, 0, 0]
    for j in i:
        index[int(j)] = 1
    cts.append(index)
    
for i in ['0','1','2','3','4']:
    name='ct_'+ i
    locals()[name]= []
    for line in cts:
        c =  np.array(line)
        locals()[name].append(c[0])
    data[name]=locals()[name]

data = data.drop(['ct'], axis=1)
print('ct has been transformed OK!')

 has been transformed OK!


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 0 to 399999
Data columns (total 37 columns):
aid                   400000 non-null int64
uid                   400000 non-null int64
label                 400000 non-null uint8
advertiserId          400000 non-null uint8
campaignId            400000 non-null uint8
creativeId            400000 non-null uint8
creativeSize          400000 non-null uint8
adCategoryId          400000 non-null uint8
productId             400000 non-null uint8
productType           400000 non-null uint8
age                   400000 non-null uint8
gender                400000 non-null uint8
marriageStatus        400000 non-null object
education             400000 non-null uint8
consumptionAbility    400000 non-null uint8
LBS                   400000 non-null uint16
interest1             400000 non-null object
interest2             400000 non-null object
interest3             400000 non-null object
interest4             400000 non-null object
int

#------------------------------分割线（底下的还没弄完）-----------------------------------------

In [ ]:
train_all_interest = X_train[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
test_all_interest = X_test[['interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
num_all_interest_train = []
num_all_interest_test = []
aids = []
train_array = np.array(train_all_interest)
test_array = np.array(test_all_interest)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_all_interest_train.append(num)

In [ ]:
data.info()